In [ ]:
# Training a vision transformer without batch normalisation and weight decay (L2 regularisation)

from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/DL project_aug1/DL project
!ls

# This model is trained using tensor flow using Keras module. the following modules are imported
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, Input, Reshape
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np
import tensorflow_hub as hub

tensorboard = TensorBoard(log_dir='./ViT', histogram_freq=0, write_graph=True, write_images=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

image_size = 224
IMG_SHAPE = (image_size, image_size, 3)

# Load the Vision Transformer model from TensorFlow Hub
vit_model_url = "https://tfhub.dev/sayakpaul/vit_b16_fe/1"
base_model = hub.KerasLayer(vit_model_url, trainable=True)

# Building the model using the Functional API
# Define the input layer
input_tensor = Input(shape=IMG_SHAPE)

# Creating a function to apply the base_model
def apply_base_model(inputs):
    return base_model(inputs, training=False) # Set training=False during inference

# Passing the input through the function to get the output
x = tf.keras.layers.Lambda(apply_base_model)(input_tensor)

# Reshaping the output of the base_model to 4D before applying GlobalAveragePooling2D
x = Reshape((1, 1, 768))(x)  # Reshape to (batch_size, 1, 1, features)

x = GlobalAveragePooling2D()(x) # Now this layer will work correctly
x = Dropout(0.4)(x) # Applying a dropout of 0.4 or 40%
output_tensor = Dense(4, activation='softmax')(x)  # Adjusting the number of classes

model = Model(inputs=input_tensor, outputs=output_tensor)  # Creating the model
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

# Preparing the data by data Augumentation usingImage data generator predefined function
train_datagen = ImageDataGenerator(rescale=0.2,
                                   rotation_range=30,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_directory('/content/gdrive/My Drive/DL project_aug1/DL project/training',
                                                    target_size=(image_size, image_size),
                                                    batch_size=32,
                                                    class_mode='sparse',
                                                    subset='training')

validation_generator = train_datagen.flow_from_directory('/content/gdrive/My Drive/DL project_aug1/DL project/testing',
                                                         target_size=(image_size, image_size),
                                                         batch_size=32,
                                                         class_mode='sparse',
                                                         subset='validation')

# Fitting the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // train_generator.batch_size,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples // validation_generator.batch_size,
                    epochs=7,
                    callbacks=[tensorboard, es])

# Evaluating the model for validating results.
scores = model.evaluate(validation_generator, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# Generation of classification report
y_pred = model.predict(validation_generator, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(validation_generator.classes, y_pred_bool, target_names=train_generator.class_indices.keys()))

# Saving the model
model.save("ViT_model.h5")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/DL project_aug1/DL project
ResNet50	       testing	 X_test.pickle	 Y_test.pickle
ResNet50accuracy87.h5  training  X_train.pickle  Y_train.pickle
Found 2297 images belonging to 4 classes.
Found 78 images belonging to 4 classes.
Epoch 1/7


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


71/71 ━━━━━━━━━━━━━━━━━━━━ 1733s 23s/step - accuracy: 0.2957 - loss: 2.5028 - val_accuracy: 0.3906 - val_loss: 1.9681
Epoch 2/7
 1/71 ━━━━━━━━━━━━━━━━━━━━ 23:31 20s/step - accuracy: 0.3438 - loss: 1.7839

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


71/71 ━━━━━━━━━━━━━━━━━━━━ 35s 208ms/step - accuracy: 0.3438 - loss: 1.7839 - val_accuracy: 0.1429 - val_loss: 2.6588
Epoch 3/7
71/71 ━━━━━━━━━━━━━━━━━━━━ 1579s 22s/step - accuracy: 0.4579 - loss: 1.6184 - val_accuracy: 0.2969 - val_loss: 2.1220
Epoch 4/7
71/71 ━━━━━━━━━━━━━━━━━━━━ 31s 140ms/step - accuracy: 0.4688 - loss: 1.6040 - val_accuracy: 0.2857 - val_loss: 1.6685
Epoch 5/7
71/71 ━━━━━━━━━━━━━━━━━━━━ 1598s 22s/step - accuracy: 0.4773 - loss: 1.5029 - val_accuracy: 0.2812 - val_loss: 2.3666
Epoch 6/7
71/71 ━━━━━━━━━━━━━━━━━━━━ 30s 116ms/step - accuracy: 0.4688 - loss: 1.5198 - val_accuracy: 0.3571 - val_loss: 1.9521
Epoch 7/7
71/71 ━━━━━━━━━━━━━━━━━━━━ 1621s 22s/step - accuracy: 0.5408 - loss: 1.3091 - val_accuracy: 0.3750 - val_loss: 2.0978
3/3 ━━━━━━━━━━━━━━━━━━━━ 51s 15s/step - accuracy: 0.2468 - loss: 2.1228
Test loss: 2.1601457595825195
Test accuracy: 0.24358974397182465
3/3 ━━━━━━━━━━━━━━━━━━━━ 52s 15s/step


              precision    recall  f1-score   support

      glioma       0.50      0.05      0.09        20
  meningioma       0.20      0.13      0.16        23
    no_tumor       0.31      0.71      0.43        21
   pituitary       0.15      0.14      0.15        14

    accuracy                           0.27        78
   macro avg       0.29      0.26      0.21        78
weighted avg       0.30      0.27      0.21        78



In [ ]:

# Modifying the model for improving accuracy using learning rate scheduler, batch normalisation, weight decay L2 regularisation

from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/DL project_aug1/DL project
!ls

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, Input, Reshape
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np
import tensorflow_hub as hub

tensorboard = TensorBoard(log_dir='./ViT', histogram_freq=0, write_graph=True, write_images=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

image_size = 224
IMG_SHAPE = (image_size, image_size, 3)

# Loading the Vision Transformer model from TensorFlow Hub
vit_model_url = "https://tfhub.dev/sayakpaul/vit_b16_fe/1"
base_model = hub.KerasLayer(vit_model_url, trainable=True)

# Build the model using the Functional API
# Define the input layer
input_tensor = Input(shape=IMG_SHAPE)

# Create a function to apply the base_model
def apply_base_model(inputs):
    return base_model(inputs, training=False) # Set training=False during inference

# Pass the input through the function to get the output
x = tf.keras.layers.Lambda(apply_base_model)(input_tensor)

# Reshape the output of the base_model to 4D before applying GlobalAveragePooling2D
x = Reshape((1, 1, 768))(x)  # Reshape to (batch_size, 1, 1, features)

x = GlobalAveragePooling2D()(x) # Now this layer will work correctly
x = Dropout(0.5)(x)  # Increase dropout rate for better regularization
output_tensor = Dense(4, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='softmax')(x)  # Adjust the number of classes and add L2 regularization

model = Model(inputs=input_tensor, outputs=output_tensor)  # Create the model
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics=['accuracy'])

# Prepare the data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_directory('/content/gdrive/My Drive/DL project_aug1/DL project/training',
                                                    target_size=(image_size, image_size),
                                                    batch_size=32,
                                                    class_mode='sparse',
                                                    subset='training')

validation_generator = train_datagen.flow_from_directory('/content/gdrive/My Drive/DL project_aug1/DL project/testing',
                                                         target_size=(image_size, image_size),
                                                         batch_size=32,
                                                         class_mode='sparse',
                                                         subset='validation')

# Fiting the model
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // train_generator.batch_size,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples // validation_generator.batch_size,
                    epochs=20,
                    callbacks=[tensorboard, es, lr_scheduler])

# Evaluate the model
scores = model.evaluate(validation_generator, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# Generate classification report
y_pred = model.predict(validation_generator, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(validation_generator.classes, y_pred_bool, target_names=train_generator.class_indices.keys()))

# Save the model
model.save("ViT_model_improved.h5")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/DL project_aug1/DL project
ResNet50	       testing	 ViT	       X_test.pickle   Y_test.pickle
ResNet50accuracy87.h5  training  ViT_model.h5  X_train.pickle  Y_train.pickle
Found 2297 images belonging to 4 classes.
Found 78 images belonging to 4 classes.
Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


71/71 ━━━━━━━━━━━━━━━━━━━━ 1633s 23s/step - accuracy: 0.2121 - loss: 4.0102 - val_accuracy: 0.2812 - val_loss: 1.8632 - learning_rate: 1.0000e-04
Epoch 2/20
 1/71 ━━━━━━━━━━━━━━━━━━━━ 24:45 21s/step - accuracy: 0.3125 - loss: 2.7345

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


71/71 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - accuracy: 0.3125 - loss: 2.7345 - val_accuracy: 0.3571 - val_loss: 2.0103 - learning_rate: 1.0000e-04
Epoch 3/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 1625s 22s/step - accuracy: 0.3141 - loss: 2.8395 - val_accuracy: 0.3125 - val_loss: 1.8880 - learning_rate: 1.0000e-04
Epoch 4/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 30s 115ms/step - accuracy: 0.4062 - loss: 2.4287 - val_accuracy: 0.0714 - val_loss: 2.3895 - learning_rate: 1.0000e-04
Epoch 5/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 1605s 22s/step - accuracy: 0.3587 - loss: 2.6508 - val_accuracy: 0.3750 - val_loss: 1.7806 - learning_rate: 5.0000e-05
Epoch 6/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 30s 113ms/step - accuracy: 0.3750 - loss: 2.6169 - val_accuracy: 0.4286 - val_loss: 2.0449 - learning_rate: 5.0000e-05
Epoch 7/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 1611s 22s/step - accuracy: 0.3751 - loss: 2.4474 - val_accuracy: 0.2812 - val_loss: 1.9658 - learning_rate: 5.0000e-05
Epoch 8/20
71/71 ━━━━━━━━━━━━━━━━━━━━ 30s 142ms/step - accuracy: 0.3750 

              precision    recall  f1-score   support

      glioma       0.36      0.20      0.26        20
  meningioma       0.20      0.17      0.19        23
    no_tumor       0.09      0.05      0.06        21
   pituitary       0.14      0.36      0.20        14

    accuracy                           0.18        78
   macro avg       0.20      0.19      0.18        78
weighted avg       0.20      0.18      0.17        78

